In [1]:
import pandas as pd

# === Step 1: Load CSV ===
file_path = "reliability_scores_matrics.csv"
df = pd.read_csv(file_path)

# === Step 2: Check column names and preview ===
print("\n📌 Available columns:")
print(df.columns.tolist())
print("\n🔍 Sample data:")
print(df.head())

# === Step 3: Define key columns ===
USER_COL = 'User ID'
ITEM_COL = 'Item ID'
SCORE_COL = 'top_ij'  # Assumed as prediction confidence
GROUND_TRUTH_COL = 'Item ID'  # Assuming current item is the ground truth

# === Step 4: Get top-K predicted items per user ===
k = 5
predicted_df = (
    df.sort_values(by=[USER_COL, SCORE_COL], ascending=[True, False])
      .groupby(USER_COL)[ITEM_COL]
      .apply(list)
      .reset_index()
      .rename(columns={ITEM_COL: 'predicted_items'})
)

# === Step 5: Merge to get true item back (assumes each user has 1 row with ground truth) ===
ground_truth_df = df[[USER_COL, ITEM_COL]].drop_duplicates(subset=USER_COL)
ground_truth_df = ground_truth_df.rename(columns={ITEM_COL: 'true_item'})

# Merge ground truth with predictions
merged_df = pd.merge(predicted_df, ground_truth_df, on=USER_COL)

# === Step 6: Compute Hit Ratio @ K ===
def hit_ratio_at_k(dataframe, k):
    hits = 0
    for _, row in dataframe.iterrows():
        if row['true_item'] in row['predicted_items'][:k]:
            hits += 1
    return hits / len(dataframe)

hr = hit_ratio_at_k(merged_df, k)
print(f"\n✅ Hit Ratio @ {k}: {hr:.4f}")

# === Step 7: Save result to CSV ===
output_path = "converted_recommendation_data.csv"
merged_df.to_csv(output_path, index=False)
print(f"\n📁 Saved processed file to: {output_path}")



📌 Available columns:
['User ID', 'Item ID', 'h_ij', 'd_ij', 'avg_score', 'review_rank', 'n_prime', 'z_ij', 'most_ij', 'sum_z', 'q_ij', 'sum_q', 'top_ij']

🔍 Sample data:
          User ID     Item ID      h_ij      d_ij  avg_score  review_rank  \
0   AO94DHGC771SJ  0528881469  0.000000  0.653309   0.156307            1   
1   AMO214LNFCEI4  0528881469  0.163296  0.202288   0.211416            2   
2  A3N7T0DY83Y4IG  0528881469  0.698923  0.089906   0.451825            3   
3  A1H8PY3QHMQQA0  0528881469  0.137781  0.040832   0.141980            4   
4  A24EV6RXELQZ63  0528881469  0.000000  0.013665   0.038471            5   

   n_prime      z_ij   most_ij     sum_z      q_ij     sum_q    top_ij  
0        5  1.000000  0.683242  1.463611  4.000000  6.416667  0.623377  
1        5  0.250000  0.170810  1.463611  1.500000  6.416667  0.233766  
2        5  0.111111  0.075916  1.463611  0.666667  6.416667  0.103896  
3        5  0.062500  0.042703  1.463611  0.250000  6.416667  0.038961  
4